In [14]:
#_*_coding:utf-8_*_
from pyspark.context import SparkContext
from pyspark.sql import SQLContext
from pyspark.context import SparkConf
from graphframes import *
conf = SparkConf().setAppName("test")
sc = SparkContext.getOrCreate(conf=conf)
spark = SQLContext(sc)
df = spark.read.csv('../book-master/data/transport-nodes.csv',header=True)

In [15]:
df.show()

+----------------+---------+---------+----------+
|              id| latitude|longitude|population|
+----------------+---------+---------+----------+
|       Amsterdam|52.379189| 4.899431|    821752|
|         Utrecht|52.092876| 5.104480|    334176|
|        Den Haag|52.078663| 4.288788|    514861|
|       Immingham| 53.61239| -0.22219|      9642|
|       Doncaster| 53.52285| -1.13116|    302400|
|Hoek van Holland|  51.9775|  4.13333|      9382|
|      Felixstowe| 51.96375|   1.3511|     23689|
|         Ipswich| 52.05917|  1.15545|    133384|
|      Colchester| 51.88921|  0.90421|    104390|
|          London|51.509865|-0.118092|   8787892|
|       Rotterdam|  51.9225|  4.47917|    623652|
|           Gouda| 52.01667|  4.70833|     70939|
+----------------+---------+---------+----------+



In [11]:
### DSL语法
nodes.printSchema()

root
 |-- id: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- population: string (nullable = true)



In [18]:
df.select("id").show()

+----------------+
|              id|
+----------------+
|       Amsterdam|
|         Utrecht|
|        Den Haag|
|       Immingham|
|       Doncaster|
|Hoek van Holland|
|      Felixstowe|
|         Ipswich|
|      Colchester|
|          London|
|       Rotterdam|
|           Gouda|
+----------------+



In [19]:
df.select(df["id"].alias("id1"),df["population"]+100).show()

+----------------+------------------+
|             id1|(population + 100)|
+----------------+------------------+
|       Amsterdam|          821852.0|
|         Utrecht|          334276.0|
|        Den Haag|          514961.0|
|       Immingham|            9742.0|
|       Doncaster|          302500.0|
|Hoek van Holland|            9482.0|
|      Felixstowe|           23789.0|
|         Ipswich|          133484.0|
|      Colchester|          104490.0|
|          London|         8787992.0|
|       Rotterdam|          623752.0|
|           Gouda|           71039.0|
+----------------+------------------+



In [20]:
df.select(df["id"].alias("id1"),df["population"]+100).toDF("id2","population2").show()

+----------------+-----------+
|             id2|population2|
+----------------+-----------+
|       Amsterdam|   821852.0|
|         Utrecht|   334276.0|
|        Den Haag|   514961.0|
|       Immingham|     9742.0|
|       Doncaster|   302500.0|
|Hoek van Holland|     9482.0|
|      Felixstowe|    23789.0|
|         Ipswich|   133484.0|
|      Colchester|   104490.0|
|          London|  8787992.0|
|       Rotterdam|   623752.0|
|           Gouda|    71039.0|
+----------------+-----------+



In [23]:
df.filter(df["population"]>300000).show()

+---------+---------+---------+----------+
|       id| latitude|longitude|population|
+---------+---------+---------+----------+
|Amsterdam|52.379189| 4.899431|    821752|
|  Utrecht|52.092876| 5.104480|    334176|
| Den Haag|52.078663| 4.288788|    514861|
|Doncaster| 53.52285| -1.13116|    302400|
|   London|51.509865|-0.118092|   8787892|
|Rotterdam|  51.9225|  4.47917|    623652|
+---------+---------+---------+----------+



In [25]:
df.groupBy("id").count().show()

+----------------+-----+
|              id|count|
+----------------+-----+
|       Doncaster|    1|
|       Rotterdam|    1|
|          London|    1|
|        Den Haag|    1|
|       Immingham|    1|
|       Amsterdam|    1|
|      Colchester|    1|
|         Utrecht|    1|
|           Gouda|    1|
|         Ipswich|    1|
|Hoek van Holland|    1|
|      Felixstowe|    1|
+----------------+-----+



In [26]:
df.agg({"population":"avg"}).show()

+---------------+
|avg(population)|
+---------------+
|      978013.25|
+---------------+



In [37]:
### 创建一张表临时表
## 1.6版本
#df.registerTempTable("city0")
### 2.2版本
##df.createTempView("city")
df.createOrReplaceTempView("city")

In [38]:
spark.sql("select * from city").show()

+----------------+---------+---------+----------+
|              id| latitude|longitude|population|
+----------------+---------+---------+----------+
|       Amsterdam|52.379189| 4.899431|    821752|
|         Utrecht|52.092876| 5.104480|    334176|
|        Den Haag|52.078663| 4.288788|    514861|
|       Immingham| 53.61239| -0.22219|      9642|
|       Doncaster| 53.52285| -1.13116|    302400|
|Hoek van Holland|  51.9775|  4.13333|      9382|
|      Felixstowe| 51.96375|   1.3511|     23689|
|         Ipswich| 52.05917|  1.15545|    133384|
|      Colchester| 51.88921|  0.90421|    104390|
|          London|51.509865|-0.118092|   8787892|
|       Rotterdam|  51.9225|  4.47917|    623652|
|           Gouda| 52.01667|  4.70833|     70939|
+----------------+---------+---------+----------+



In [48]:
### 创建全局的临时视图 不同的spark session中均可访问
##df.createGlobalTempView
df.createOrReplaceGlobalTempView("city2")
spark.sql("select * from global_temp.city2").show()

+----------------+---------+---------+----------+
|              id| latitude|longitude|population|
+----------------+---------+---------+----------+
|       Amsterdam|52.379189| 4.899431|    821752|
|         Utrecht|52.092876| 5.104480|    334176|
|        Den Haag|52.078663| 4.288788|    514861|
|       Immingham| 53.61239| -0.22219|      9642|
|       Doncaster| 53.52285| -1.13116|    302400|
|Hoek van Holland|  51.9775|  4.13333|      9382|
|      Felixstowe| 51.96375|   1.3511|     23689|
|         Ipswich| 52.05917|  1.15545|    133384|
|      Colchester| 51.88921|  0.90421|    104390|
|          London|51.509865|-0.118092|   8787892|
|       Rotterdam|  51.9225|  4.47917|    623652|
|           Gouda| 52.01667|  4.70833|     70939|
+----------------+---------+---------+----------+



In [51]:
### RDD与DataFrame互操作
rdd = sc.textFile("./data/wc1.txt").flatMap(lambda x:x.split(" ")).map(lambda x:(x,1)).reduceByKey(lambda x,y:x+y)

In [53]:
rdd.collect()

[('hello', 4), ('world', 1), ('Ireland', 1), ('ML', 1), ('Neo4j', 1)]

In [57]:
df_temp = rdd.toDF(["word","cnt"])

In [58]:
df_temp.show()

+-------+---+
|   word|cnt|
+-------+---+
|  hello|  4|
|  world|  1|
|Ireland|  1|
|     ML|  1|
|  Neo4j|  1|
+-------+---+



In [59]:
###通过schemach创建dataframe
from pyspark.sql import Row
w_cnt = rdd.map(lambda x:Row(word = x[0],cnt = x[1]))
df_temp2 = spark.createDataFrame(w_cnt)

In [61]:
df_temp2.show()

+---+-------+
|cnt|   word|
+---+-------+
|  4|  hello|
|  1|  world|
|  1|Ireland|
|  1|     ML|
|  1|  Neo4j|
+---+-------+



In [83]:
df_temp2.dtypes

[('cnt', 'bigint'), ('word', 'string')]

In [76]:
### 定义schema的type
from pyspark.sql.types import StructType,StructField,StringType,IntegerType
schema = StructType([StructField("word",StringType()),StructField("cnt",IntegerType())])
df_temp3 = spark.createDataFrame(w_cnt,schema)
df_temp3.show()
## spark.read.schema(schema).json(path)
#### spark.read.schema(schema).load(path)

+-------+---+
|   word|cnt|
+-------+---+
|  hello|  4|
|  world|  1|
|Ireland|  1|
|     ML|  1|
|  Neo4j|  1|
+-------+---+



In [85]:
df_temp3.dtypes

[('word', 'string'), ('cnt', 'int')]

In [86]:
### sparksql和hive集成，需要将hive-site.xml文件和mysql的驱动包放到spark_home中的conf和jars里面
### 如果配置了metastore服务，必须还要开启metastore